In [2]:
import json
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import nibabel as nib
from typing import List
from collections import OrderedDict
import SimpleITK as sitk
from tqdm import notebook

In [ ]:
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [1]:
# !pip install nibabel
# !pip install SimpleITK

Looking in indexes: http://ftp.daumkakao.com/pypi/simple
     |████████████████████████████████| 3.3 MB 12.1 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Looking in indexes: http://ftp.daumkakao.com/pypi/simple
     |████████████████████████████████| 48.4 MB 9.3 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [3]:
# Utils ..
def maybe_mkdir_p(directory: str) -> None:
    os.makedirs(directory, exist_ok=True)
    
def subdirs(folder: str, join: bool = True, prefix: str = None, suffix: str = None, sort: bool = True) -> List[str]:
    if join:
        l = os.path.join
    else:
        l = lambda x, y: y
    res = [l(folder, i) for i in os.listdir(folder) if os.path.isdir(os.path.join(folder, i))
           and (prefix is None or i.startswith(prefix))
           and (suffix is None or i.endswith(suffix))]
    if sort:
        res.sort()
    return res

def subfiles(folder: str, join: bool = True, prefix: str = None, suffix: str = None, sort: bool = True) -> List[str]:
    if join:
        l = os.path.join
    else:
        l = lambda x, y: y
    res = [l(folder, i) for i in os.listdir(folder) if os.path.isfile(os.path.join(folder, i))
           and (prefix is None or i.startswith(prefix))
           and (suffix is None or i.endswith(suffix))]
    if sort:
        res.sort()
    return res

In [4]:
maic_dir = '/tf/backup/'
base_dir = os.path.join(maic_dir, 'working')
training_folder = '/mnt/dataset/train-valid_set'
test_folder = '/mnt/dataset/test_set'
# input_dir = '/mnt/dataset/train-valid_set/Graspers (straight) (OLYMPUS) (GRSL-UCOL)/'
temp_dir = '/tf/temp/'

maybe_mkdir_p(base_dir)
maybe_mkdir_p(temp_dir)

# ! git clone https://github.com/keemsir/nnUNet.git

respository_dir = os.path.join(maic_dir, 'nnUNet')
os.chdir(respository_dir)

! pip install -e .

os.chdir(base_dir)

Looking in indexes: http://ftp.daumkakao.com/pypi/simple
Obtaining file:///tf/backup/nnUNet
     |████████████████████████████████| 151 kB 15.2 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 35.3 MB/s eta 0:00:01
     |████████████████████████████████| 2.0 MB 20.3 MB/s eta 0:00:01
     |████████████████████████████████| 829 kB 127.6 MB/s eta 0:00:01
     |████████████████████████████████| 96 kB 122.3 MB/s eta 0:00:01
  Created wheel for dicom2nifti: filename=dicom2nifti-2.3.0-py3-none-any.whl size=42933 sha256=ca1de8bfe41beb690ff7814651a123d2d3e50358ae71f774f24c08c2eb941693
  Stored in directory: /root/.cache/pip/wheels/6c/63/4b/bfdb925ebf1c6ee721258d7f8a69951cd76e6d178bb0d0395b
  Created wheel for medpy: filename=MedPy-0.4.0-py3-none-any.whl size=214963 sha256=62e2fe88725ed4d095536818b8b46ffa31b3d31e176110240e1c23b7afdbc428
  Stored in directory: /root/.cache/pip/wheels/59/90/13/5b5a86494a571d07e6549b9910a7bdc9728aa15b4778dfddff
  Created wheel for batchgenerators: fil

In [5]:
task_name = 'Task81_NDDR'
'''
Task80_GRSR
Task81_NDDR
Task82_BFCR
Task83_MSCR
Task84_SIRL
Task85_GRSL
Task86_NDHL
Task87_CLALM3MT
Task88_CLALUCUK
'''

main_dir = os.path.join(base_dir, 'nnUNet/nnunet')
mainT_dir = os.path.join(temp_dir, 'nnUNet/nnunet')

rawbase_dir = os.path.join(mainT_dir, 'nnUNet_raw_data_base/')

pp_dir = os.path.join(mainT_dir, 'preprocessed')
tasks_dir = os.path.join(mainT_dir, 'Tasks')
task_dir = os.path.join(tasks_dir, task_name)

model_dir = os.path.join(main_dir, 'nnUNet_trained_models')

# 1. Data preprocessing
maybe_mkdir_p(tasks_dir)
maybe_mkdir_p(temp_dir)

# 2. Directory
maybe_mkdir_p(main_dir)
maybe_mkdir_p(model_dir)
maybe_mkdir_p(pp_dir)

# 3. Directory
# maybe_mkdir_p(result_dir)
# maybe_mkdir_p(staple_dir)


#Environment Setting
os.environ['nnUNet_raw_data_base'] = rawbase_dir #os.path.join(mainT_dir, 'nnUNet_raw_data_base')
os.environ['nnUNet_preprocessed'] = pp_dir #os.path.join(mainT_dir, 'preprocessed')
os.environ['RESULTS_FOLDER'] = model_dir #os.path.join(main_dir, 'nnUNet_trained_models')


print('Setting Completed!')

Setting Completed!


# 1-1. Data preprocessing

In [7]:
'''# jsonPath : JSON Path

f_json = open(jsonPath, 'rt', encoding='UTF8')
jsonData =json.load(f_json)
jsonPoints= jsonData.get('annotations')[0].get('points')
jsonImgSize = jsonData.get('images')


points = np.array(np.round(jsonPoints), dtype=np.int)
mask = np.zeros((jsonImgSize.get('height'), jsonImgSize.get('width')), dtype = np.uint8)
cv2.fillPoly(mask,[points], 1)'''

"# jsonPath : JSON Path\n\nf_json = open(jsonPath, 'rt', encoding='UTF8')\njsonData =json.load(f_json)\njsonPoints= jsonData.get('annotations')[0].get('points')\njsonImgSize = jsonData.get('images')\n\n\npoints = np.array(np.round(jsonPoints), dtype=np.int)\nmask = np.zeros((jsonImgSize.get('height'), jsonImgSize.get('width')), dtype = np.uint8)\ncv2.fillPoly(mask,[points], 1)"

In [45]:
Group_frame(2)

Print surgery : R-Maryland (da Vinci) (BFCR-MADV)


['SNUH_DC07_JCW0_RLPN_0003',
 'SNUH_DC07_JCW0_RALP_0000',
 'SNUH_DC08_KHS0_ENME_0032',
 'SNUH_DC07_JCW0_RALP_0031',
 'SNUH_DC07_JCW0_RLPN_0008',
 'SNUH_DC07_JCW0_RALP_0001',
 'SNUH_DC07_JCW0_RLPN_0009',
 'SNUH_DC07_JCW0_RLPN_0002',
 'SNUH_DC07_JCW0_RALP_0021',
 'SNUH_DC07_JCW0_RLPN_0014',
 'SNUH_DC07_JCW0_RALP_0011']

In [28]:
for i in subfiles('/mnt/dataset/train-valid_set/Graspers (straight) (OLYMPUS) (GRSL-UCOL)/',prefix=Group_frame(5)[6], suffix='.png'):
    plt_i = plt.imread(i)
    print(plt_i.shape)

Print surgery : Graspers (straight) (OLYMPUS) (GRSL-UCOL)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)
(1080, 1920, 3)


In [6]:
def Group_frame(surgery_NUM: int, surgery_path = '/mnt/dataset/train-valid_set'):

    '''
    surgery_path(default) : '/mnt/dataset/train-valid_set/'
    surgery_NUM category : 
    '''
    
    surgery_DIC = {0: 'Prograsp (da Vinci) (GRSR-UCDV)',
                   1: 'R-LND (da Vinci) (NDDR-LADV)',
                   2: 'R-Maryland (da Vinci) (BFCR-MADV)',
                   3: 'R-Scissors (da Vinci) (MSCR-UCDV)',
                   4: 'Suction Irrigator (SIRL-UCUK)',
                   5: 'Graspers (straight) (OLYMPUS) (GRSL-UCOL)',
                   6: 'Needle Holder (AESCULAP) (NDHL-UCAE)',
                   7: 'Metal Clip Applier (Medtronic) (CLAL-M3MT)',
                   8: 'Polymer Clip Applier (CLAL-UCUK)'}

    print('Print surgery : ' + (surgery_DIC[surgery_NUM]))

    SG_listpath = os.path.join(surgery_path, surgery_DIC[surgery_NUM])
    SG_listname = subfiles(SG_listpath, join=False, suffix='.json')
    
    Frame_class = []
    
    for sg in SG_listname:
        name_, ext_ = os.path.splitext(sg)
        name1, name2, name3, name4, name5, name6 = name_.split('_')
        Frame_class.append('{}_{}_{}_{}_{}'.format(name1, name2, name3, name4, name5))
    
    set_class = list(set(Frame_class))

    return set_class

#
# Group_frame(1)

In [8]:
def png2nifti(surgery_NUM: int, 
              training_folder: str = '/mnt/dataset/train-valid_set', test_folder: str = '/mnt/dataset/test_set',
              save_folder: str = '/tf/temp/nnUNet/nnunet/Tasks/'):

    # surgery_NUM : surgery_DIC Num (0~8)
    # training_folder : training file path '/mnt/dataset/train-valid_set'
    # test_folder : test file path '/mnt/dataset/test_set'
    # save_folder : '/tf/temp/nnUNet/nnunet/Tasks/' [imagesTr, imagesTs] Save Folder path

    # Surgery Catalog
    surgery_DIC = {0: 'Prograsp (da Vinci) (GRSR-UCDV)',
                   1: 'R-LND (da Vinci) (NDDR-LADV)',
                   2: 'R-Maryland (da Vinci) (BFCR-MADV)',
                   3: 'R-Scissors (da Vinci) (MSCR-UCDV)',
                   4: 'Suction Irrigator (SIRL-UCUK)',
                   5: 'Graspers (straight) (OLYMPUS) (GRSL-UCOL)',
                   6: 'Needle Holder (AESCULAP) (NDHL-UCAE)',
                   7: 'Metal Clip Applier (Medtronic) (CLAL-M3MT)',
                   8: 'Polymer Clip Applier (CLAL-UCUK)'}
    
    task_NameDIC = {0: 'Task80_GRSR', 
                    1: 'Task81_NDDR', 
                    2: 'Task82_BFCR', 
                    3: 'Task83_MSCR', 
                    4: 'Task84_SIRL', 
                    5: 'Task85_GRSL', 
                    6: 'Task86_NDHL', 
                    7: 'Task87_CLALM3MT', 
                    8: 'Task88_CLALUCUK'}


    maybe_mkdir_p(os.path.join(save_folder, task_NameDIC[surgery_NUM], 'imagesTr'))
    maybe_mkdir_p(os.path.join(save_folder, task_NameDIC[surgery_NUM], 'imagesTs'))
    maybe_mkdir_p(os.path.join(save_folder, task_NameDIC[surgery_NUM], 'labelsTr'))
    
    print('Creating "{}" Image & Label ..'.format(os.path.basename(os.path.normpath(task_NameDIC[surgery_NUM]))))


    for gf in notebook.tqdm(Group_frame(surgery_NUM)):


        training_files = subfiles(os.path.join(training_folder, surgery_DIC[surgery_NUM]), join=False, prefix=gf, suffix='.json')
        
        # Remove Item (surgery_NUMM = 2)

        try:
            training_files.remove('SNUH_DC07_JCW0_RLPN_0003_0038971.json')
        
        except:
            pass


        # Initial Setting

        images = np.zeros([512, 512, 0], dtype=np.single)

        f_json = open(os.path.join(training_folder, surgery_DIC[surgery_NUM], training_files[0]), 'rt', encoding='UTF8')  # temp training_files[0]
        jsonData = json.load(f_json)
        jsonImgSize = jsonData.get('images')
        json_height = jsonImgSize.get('height')
        json_width = jsonImgSize.get('width')

        labels = np.zeros([512, 512, 0], dtype=np.uint8)



        for training_file in training_files: # training_files : JSON file list

            training_file_name = os.path.splitext(training_file)[0]  # split suffix, Only Name
            
            try:
                # json label
                f_json = open(os.path.join(training_folder, surgery_DIC[surgery_NUM], training_file), 'rt', encoding='UTF8')
                jsonData = json.load(f_json)
                jsonPoints = jsonData.get('annotations')[0].get('points')
                jsonImgSize = jsonData.get('images')
                
                points = np.array(np.round(jsonPoints), dtype=np.int_)
                json_height = jsonImgSize.get('height')
                json_width = jsonImgSize.get('width')
                label_json = np.zeros((json_height, json_width), dtype=np.uint8)
                cv2.fillPoly(label_json, [points], 1)
                label_json = cv2.resize(label_json, dsize=(512, 512), interpolation=cv2.INTER_CUBIC) # reduce
            except:
                print('Error file :', training_file)


            # png image
            
            
            training_path = os.path.join(training_folder, surgery_DIC[surgery_NUM], training_file)
            png_data = plt.imread(os.path.join(training_folder, training_path[:-5] + '.png'))
            png_data = np.mean(png_data, axis=2)
            png_data = cv2.resize(png_data, dsize=(512, 512), interpolation=cv2.INTER_CUBIC) # size reduce


            image_png = np.expand_dims(png_data, axis=2)

            images = np.append(images, image_png, axis=2)

            label_json = np.expand_dims(label_json, axis=2)

            labels = np.append(labels, label_json, axis=2)


        niim = nib.Nifti1Image(images, affine=np.eye(4))
        nib.save(niim, os.path.join(save_folder, task_NameDIC[surgery_NUM], 'imagesTr/{}.nii.gz'.format(gf)))
        print(images.shape)
        del images

        nila = nib.Nifti1Image(labels, affine=np.eye(4))
        nib.save(nila, os.path.join(save_folder, task_NameDIC[surgery_NUM], 'labelsTr/{}.nii.gz'.format(gf)))

        del labels


    print('"{}" Image & Label Completed !!'.format(os.path.basename(os.path.normpath(task_NameDIC[surgery_NUM]))))

In [26]:
png2nifti(surgery_NUM=3)

Creating "Task83_MSCR" Image & Label ..
Print surgery : R-Scissors (da Vinci) (MSCR-UCDV)


  0%|          | 0/12 [00:00<?, ?it/s]

(512, 512, 262)
(512, 512, 253)
(512, 512, 282)
(512, 512, 145)
(512, 512, 253)
(512, 512, 166)
(512, 512, 697)
(512, 512, 161)
(512, 512, 166)
(512, 512, 174)
(512, 512, 128)
(512, 512, 328)
"Task83_MSCR" Image & Label Completed !!


In [56]:
# os.listdir('/mnt/dataset/train-valid_set/R-Maryland (da Vinci) (BFCR-MADV)/SNUH_DC07_JCW0_RLPN')
subfiles('/mnt/dataset/train-valid_set/R-Maryland (da Vinci) (BFCR-MADV)/', prefix='SNUH_DC07_JCW0_RLPN_0003_0038971')

['/mnt/dataset/train-valid_set/R-Maryland (da Vinci) (BFCR-MADV)/SNUH_DC07_JCW0_RLPN_0003_0038971.json',
 '/mnt/dataset/train-valid_set/R-Maryland (da Vinci) (BFCR-MADV)/SNUH_DC07_JCW0_RLPN_0003_0038971.xml']

In [16]:
def json_mk(save_dir: str):
    # Path
    imagesTr = os.path.join(save_dir, 'imagesTr')
    imagesTs = os.path.join(save_dir, 'imagesTs')
    maybe_mkdir_p(imagesTr)
    maybe_mkdir_p(imagesTs)
    
    overwrite_json_file = True
    json_file_exist = False
    
    if os.path.exists(os.path.join(save_dir, 'dataset.json')):
        print('dataset.json already exist!')
        json_file_exist = True
        
    if json_file_exist == False or overwrite_json_file:
        
        json_dict = OrderedDict()
        json_dict['name'] = "SUR"
        json_dict['description'] = "Robotic Surgery AI Challenge 2021"
        json_dict['tensorImageSize'] = "3D"
        json_dict['reference'] = "https://maic.or.kr/"
        json_dict['licence'] = "SNUH"
        json_dict['release'] = "24/11/2021"
        
        json_dict['modality'] = {
            "0": "png"
        }
        json_dict['labels'] = {
            "0": "background",
            "1": "surgery"
        }

        train_ids = subfiles(imagesTr, join=False, suffix='.nii.gz')
        test_ids = subfiles(imagesTs, join=False, suffix='.nii.gz')
        json_dict['numTraining'] = len(train_ids)
        json_dict['numTest'] = len(test_ids)

        json_dict['training'] = [{'image': "./imagesTr/%s" % i, "label": "./labelsTr/%s" % i} for i in train_ids]

        json_dict['test'] = ["./imagesTs/%s" % i for i in test_ids] #(i[:i.find("_0000")])

        with open(os.path.join(save_dir, "dataset.json"), 'w') as f:
            json.dump(json_dict, f, indent=4, sort_keys=False)

        if os.path.exists(os.path.join(save_dir, 'dataset.json')):
            if json_file_exist == False:
                print('{}/dataset.json <-- created!'.format(save_dir))
            else:
                print('{}/dataset.json <-- overwritten!'.format(save_dir))



In [27]:
os.listdir('/tf/temp/nnUNet/nnunet/Tasks/')

['Task82_BFCR', 'Task99_Inference', '.ipynb_checkpoints', 'Task83_MSCR']

In [28]:
json_mk('/tf/temp/nnUNet/nnunet/Tasks/Task83_MSCR')

/tf/temp/nnUNet/nnunet/Tasks/Task83_MSCR/dataset.json <-- created!


# 1-2. Data preprocessing

In [29]:

!nnUNet_convert_decathlon_task -i /tf/temp/nnUNet/nnunet/Tasks/Task83_MSCR -output_task_id 483 # -i : task_dir
!nnUNet_plan_and_preprocess -t 483 # --verify_dataset_integrity




Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

SNUH_DC07_JCW0_RALP_0000
SNUH_DC07_JCW0_RALP_0001
SNUH_DC07_JCW0_RALP_0011
SNUH_DC07_JCW0_RALP_0021
SNUH_DC07_JCW0_RALP_0031
SNUH_DC07_JCW0_RLPN_0002
SNUH_DC07_JCW0_RLPN_0003
SNUH_DC07_JCW0_RLPN_0008
before crop: (1, 128, 512, 512) after crop: (1, 128, 512, 512) spacing: [1. 

In [30]:
!nnUNet_train 2d nnUNetTrainerV2 483 all



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

###############################################
I am running the following nnUNet: 2d
My trainer class is:  <class 'nnunet.training.network_training.nnUNetTrainerV2.nnUNetTrainerV2'>
For that I will be using the following configuration:
num_classes:  1
modalities:  {0: 'png'}
use_mask_for_norm OrderedDict([(0, False)])
keep_only_largest_region None
min_region_size_per_class None
min_size_per_class None
normalization_schemes OrderedDict([(0, 'nonCT')])
stages...

stage:  0
{'batch_size': 12, 'num_pool_per_axis': [7, 7], 'patch_size': array([512, 512]), 'median_patient_size_in_voxels': array([214, 512, 512]), 'current_spacing': arra

In [12]:
png2nifti(surgery_NUM=5)

Creating "Task85_GRSL" Image & Label ..
Print surgery : Graspers (straight) (OLYMPUS) (GRSL-UCOL)


  0%|          | 0/10 [00:00<?, ?it/s]

(512, 512, 9)
(512, 512, 129)
(512, 512, 48)
(512, 512, 116)
(512, 512, 8)
(512, 512, 10)
(512, 512, 81)
(512, 512, 145)
(512, 512, 79)
(512, 512, 180)
"Task85_GRSL" Image & Label Completed !!


In [13]:
json_mk('/tf/temp/nnUNet/nnunet/Tasks/Task85_GRSL')

/tf/temp/nnUNet/nnunet/Tasks/Task85_GRSL/dataset.json <-- created!


In [14]:
!nnUNet_convert_decathlon_task -i /tf/temp/nnUNet/nnunet/Tasks/Task85_GRSL -output_task_id 485 # -i : task_dir
!nnUNet_plan_and_preprocess -t 485 # --verify_dataset_integrity



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

SNUH_DC07_JCW0_RALP_0000
SNUH_DC07_JCW0_RALP_0001
SNUH_DC07_JCW0_RALP_0008
SNUH_DC07_JCW0_RALP_0010
SNUH_DC07_JCW0_RALP_0011
SNUH_DC07_JCW0_RALP_0012
SNUH_DC07_JCW0_RALP_0013
SNUH_DC07_JCW0_RALP_0020
before crop: (1, 8, 512, 512) after crop: (1, 8, 512, 512) spacing: [1. 1. 1

In [15]:
!nnUNet_train 2d nnUNetTrainerV2 485 all



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

###############################################
I am running the following nnUNet: 2d
My trainer class is:  <class 'nnunet.training.network_training.nnUNetTrainerV2.nnUNetTrainerV2'>
For that I will be using the following configuration:
num_classes:  1
modalities:  {0: 'png'}
use_mask_for_norm OrderedDict([(0, False)])
keep_only_largest_region None
min_region_size_per_class None
min_size_per_class None
normalization_schemes OrderedDict([(0, 'nonCT')])
stages...

stage:  0
{'batch_size': 12, 'num_pool_per_axis': [7, 7], 'patch_size': array([512, 512]), 'median_patient_size_in_voxels': array([ 80, 512, 512]), 'current_spacing': arra

In [16]:
png2nifti(surgery_NUM=6)

Creating "Task86_NDHL" Image & Label ..
Print surgery : Needle Holder (AESCULAP) (NDHL-UCAE)


  0%|          | 0/38 [00:00<?, ?it/s]

(512, 512, 16)
(512, 512, 71)
(512, 512, 11)
(512, 512, 75)
(512, 512, 27)
(512, 512, 9)
(512, 512, 46)
(512, 512, 6)
(512, 512, 17)
(512, 512, 82)
(512, 512, 4)
(512, 512, 3)
(512, 512, 70)
(512, 512, 28)
(512, 512, 1)
(512, 512, 6)
(512, 512, 9)
(512, 512, 12)
(512, 512, 9)
(512, 512, 16)
(512, 512, 29)
(512, 512, 23)
(512, 512, 19)
(512, 512, 3)
(512, 512, 12)
(512, 512, 28)
(512, 512, 27)
(512, 512, 32)
(512, 512, 8)
(512, 512, 7)
(512, 512, 16)
(512, 512, 1)
(512, 512, 3)
(512, 512, 25)
(512, 512, 100)
(512, 512, 43)
(512, 512, 52)
(512, 512, 32)
"Task86_NDHL" Image & Label Completed !!


In [17]:
json_mk('/tf/temp/nnUNet/nnunet/Tasks/Task86_NDHL')

/tf/temp/nnUNet/nnunet/Tasks/Task86_NDHL/dataset.json <-- created!


In [18]:
!nnUNet_convert_decathlon_task -i /tf/temp/nnUNet/nnunet/Tasks/Task86_NDHL -output_task_id 486 # -i : task_dir
!nnUNet_plan_and_preprocess -t 486 # --verify_dataset_integrity



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

SNUH_DC07_JCW0_RALP_0000
SNUH_DC07_JCW0_RALP_0002
SNUH_DC07_JCW0_RALP_0020
SNUH_DC07_JCW0_RALP_0031
SNUH_DC16_KSH0_LDG0_0032
SNUH_DC16_KSH0_LDG0_0034
SNUH_DC16_KSH0_LDG0_0036
SNUH_DC16_KSH0_LDG0_0038
before crop: (1, 3, 512, 512) after crop: (1, 3, 512, 512) spacing: [1. 1. 1

In [19]:
!nnUNet_train 2d nnUNetTrainerV2 486 all



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

###############################################
I am running the following nnUNet: 2d
My trainer class is:  <class 'nnunet.training.network_training.nnUNetTrainerV2.nnUNetTrainerV2'>
For that I will be using the following configuration:
num_classes:  1
modalities:  {0: 'png'}
use_mask_for_norm OrderedDict([(0, False)])
keep_only_largest_region None
min_region_size_per_class None
min_size_per_class None
normalization_schemes OrderedDict([(0, 'nonCT')])
stages...

stage:  0
{'batch_size': 12, 'num_pool_per_axis': [7, 7], 'patch_size': array([512, 512]), 'median_patient_size_in_voxels': array([ 16, 512, 512]), 'current_spacing': arra

In [48]:
!nnUNet_train 3d_fullres nnUNetTrainerV2 480 all



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

###############################################
I am running the following nnUNet: 3d_fullres
My trainer class is:  <class 'nnunet.training.network_training.nnUNetTrainerV2.nnUNetTrainerV2'>
For that I will be using the following configuration:
num_classes:  1
modalities:  {0: 'png'}
use_mask_for_norm OrderedDict([(0, False)])
keep_only_largest_region None
min_region_size_per_class None
min_size_per_class None
normalization_schemes OrderedDict([(0, 'nonCT')])
stages...

stage:  0
{'batch_size': 2, 'num_pool_per_axis': [3, 4, 4], 'patch_size': array([ 40, 256, 240]), 'median_patient_size_in_voxels': array([ 67, 380, 380]), 'current

In [52]:
!nnUNet_train 2d nnUNetTrainerV2_Loss_Dice 480 all



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

###############################################
I am running the following nnUNet: 2d
My trainer class is:  <class 'nnunet.training.network_training.nnUNet_variants.loss_function.nnUNetTrainerV2_Loss_Dice.nnUNetTrainerV2_Loss_Dice'>
For that I will be using the following configuration:
num_classes:  1
modalities:  {0: 'png'}
use_mask_for_norm OrderedDict([(0, False)])
keep_only_largest_region None
min_region_size_per_class None
min_size_per_class None
normalization_schemes OrderedDict([(0, 'nonCT')])
stages...

stage:  0
{'batch_size': 12, 'num_pool_per_axis': [7, 7], 'patch_size': array([512, 512]), 'median_patient_size_in_voxels

In [53]:
!nnUNet_train 2d nnUNetTrainerV2_Loss_CE 480 all



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

###############################################
I am running the following nnUNet: 2d
My trainer class is:  <class 'nnunet.training.network_training.nnUNet_variants.loss_function.nnUNetTrainerV2_Loss_CE.nnUNetTrainerV2_Loss_CE'>
For that I will be using the following configuration:
num_classes:  1
modalities:  {0: 'png'}
use_mask_for_norm OrderedDict([(0, False)])
keep_only_largest_region None
min_region_size_per_class None
min_size_per_class None
normalization_schemes OrderedDict([(0, 'nonCT')])
stages...

stage:  0
{'batch_size': 12, 'num_pool_per_axis': [7, 7], 'patch_size': array([512, 512]), 'median_patient_size_in_voxels': a

In [54]:
!nnUNet_train 2d nnUNetTrainerV2_Loss_DiceTopK10 480 all



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

###############################################
I am running the following nnUNet: 2d
My trainer class is:  <class 'nnunet.training.network_training.nnUNet_variants.loss_function.nnUNetTrainerV2_Loss_DiceTopK10.nnUNetTrainerV2_Loss_DiceTopK10'>
For that I will be using the following configuration:
num_classes:  1
modalities:  {0: 'png'}
use_mask_for_norm OrderedDict([(0, False)])
keep_only_largest_region None
min_region_size_per_class None
min_size_per_class None
normalization_schemes OrderedDict([(0, 'nonCT')])
stages...

stage:  0
{'batch_size': 12, 'num_pool_per_axis': [7, 7], 'patch_size': array([512, 512]), 'median_patient_si

In [55]:
!nnUNet_train 2d nnUNetTrainerV2_focalLoss 480 all



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

###############################################
I am running the following nnUNet: 2d
My trainer class is:  None
For that I will be using the following configuration:
num_classes:  1
modalities:  {0: 'png'}
use_mask_for_norm OrderedDict([(0, False)])
keep_only_largest_region None
min_region_size_per_class None
min_size_per_class None
normalization_schemes OrderedDict([(0, 'nonCT')])
stages...

stage:  0
{'batch_size': 12, 'num_pool_per_axis': [7, 7], 'patch_size': array([512, 512]), 'median_patient_size_in_voxels': array([ 90, 512, 512]), 'current_spacing': array([1., 1., 1.]), 'original_spacing': array([1., 1., 1.]), 'pool_op_ker

In [56]:
!nnUNet_train 2d nnUNetTrainerV2_Loss_MCC 480 all



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

###############################################
I am running the following nnUNet: 2d
My trainer class is:  <class 'nnunet.training.network_training.nnUNet_variants.loss_function.nnUNetTrainerV2_Loss_MCC.nnUNetTrainerV2_Loss_MCC'>
For that I will be using the following configuration:
num_classes:  1
modalities:  {0: 'png'}
use_mask_for_norm OrderedDict([(0, False)])
keep_only_largest_region None
min_region_size_per_class None
min_size_per_class None
normalization_schemes OrderedDict([(0, 'nonCT')])
stages...

stage:  0
{'batch_size': 12, 'num_pool_per_axis': [7, 7], 'patch_size': array([512, 512]), 'median_patient_size_in_voxels':

In [57]:
!nnUNet_train 2d nnUNetTrainerV2_Loss_TopK10 480 all



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

###############################################
I am running the following nnUNet: 2d
My trainer class is:  <class 'nnunet.training.network_training.nnUNet_variants.loss_function.nnUNetTrainerV2_Loss_TopK10.nnUNetTrainerV2_Loss_TopK10'>
For that I will be using the following configuration:
num_classes:  1
modalities:  {0: 'png'}
use_mask_for_norm OrderedDict([(0, False)])
keep_only_largest_region None
min_region_size_per_class None
min_size_per_class None
normalization_schemes OrderedDict([(0, 'nonCT')])
stages...

stage:  0
{'batch_size': 12, 'num_pool_per_axis': [7, 7], 'patch_size': array([512, 512]), 'median_patient_size_in_vo

# 2. Training

In [33]:
# Model 1 (epoch 50)

!nnUNet_train 2d nnUNetTrainerV2 480 0
!nnUNet_train 2d nnUNetTrainerV2 480 1
!nnUNet_train 2d nnUNetTrainerV2 480 2
!nnUNet_train 2d nnUNetTrainerV2 480 3
!nnUNet_train 2d nnUNetTrainerV2 480 4



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

Traceback (most recent call last):
  File "/usr/local/bin/nnUNet_train", line 33, in <module>
    sys.exit(load_entry_point('nnunet', 'console_scripts', 'nnUNet_train')())
  File "/tf/backup/nnUNet/nnunet/run/run_training.py", line 137, in main
    trainer_class = get_default_configuration(network, task, network_trainer, plans_identifier)
  File "/tf/backup/nnUNet/nnunet/run/default_configuration.py", line 47, in get_default_configuration
    plans = load_pickle(plans_file)
  File "/usr/local/lib/python3.8/dist-packages/batchgenerators/utilities/file_and_folder_operations.py", line 57, in load_pickle
    with open(file, mode) as f